In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-fnx3z0dt
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-fnx3z0dt
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=045203127f43cc3057952d6318fb84657670fae528f2d5f2af05ea10bbe3d76a
  Stored in directory: /tmp/pip-ephem-wheel-cache-zz28fpsw/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu

#include <stdio.h>
#include <time.h>
#define BLOCK_SIZE 32
#define RADIUS 1

__host__ void init_vector(int** v, int n, int val);
__host__ void operate(int* A, int* B, int n);
__host__ bool check_result(int* A, int* B, int n);
__global__ void reduce(int* A, int* B, int n);

int main(int argc, char** argv)
{
    srand(time(NULL));

    int n = (1 << 4),
        *A = nullptr,
        *B = nullptr;

    init_vector(&A, n + 2, 0);
    init_vector(&B, n, 0);

    operate(A, B, n);

    free(B);
    free(A);

    return 0;
}

__host__ void init_vector(int** v, int n, int val)
{
    *v = (int*)malloc(sizeof(int) * n);
    for(int i = 0; i < n; (*v)[i++]=i);
    for(int i = 0; i < n; i++)
      printf("%d ", (*v)[i]);
    printf("\n");
}

__host__ void operate(int* A, int* B, int n)
{
    int* dev_A, *dev_B;

    size_t count_A = sizeof(int) * (n + 2), count_B = sizeof(int) * n;
    cudaError_t err;

    err = cudaMalloc(&dev_A, count_A);
    if(err)
      printf("1A %s\n", cudaGetErrorString(err));

    err = cudaMalloc(&dev_B, count_B);
    if(err)
      printf("1B %s\n", cudaGetErrorString(err));

    err = cudaMemcpy(dev_A, A, count_A, cudaMemcpyHostToDevice);
    if(err)
      printf("2 %s\n", cudaGetErrorString(err));

    reduce<<<n / BLOCK_SIZE + 1, BLOCK_SIZE>>>(dev_A, dev_B, 18);

    err = cudaMemcpy(B, dev_B, count_B, cudaMemcpyDeviceToHost);
    if(err)
      printf("3 %s\n", cudaGetErrorString(err));

    for(int i = 0; i < n; i++)
      printf("%d ", B[i]);

    cudaFree(dev_B);
    cudaFree(dev_A);
}

__global__ void reduce(int* A, int* B, int n)
{
    __shared__ int sh_in[BLOCK_SIZE];
    int gidx = threadIdx.x + blockIdx.x * blockDim.x;
    int lidx = threadIdx.x + RADIUS;

    // Load data into shared memory
    if (gidx < n) {
        sh_in[lidx] = A[gidx];
        if (threadIdx.x < RADIUS) {

            sh_in[lidx - RADIUS] = A[gidx];
            sh_in[lidx + BLOCK_SIZE] = A[gidx + BLOCK_SIZE];
        }
    }
    __syncthreads();

    // Apply stencil
    int sum = 0;
    if (gidx < n) {
            sum = sh_in[lidx]+sh_in[lidx + 1]+sh_in[lidx + 2];
        B[gidx] = sum;
    }
}


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 
3 6 9 12 15 18 21 24 27 30 33 36 39 42 45 48 
